# Fake News Detector
##### The goal of this task is to practice your coding skills creating a very interesting system: A Fake News Detector. So, here you have a code snippet that will guide you through the developing of the whole system. Good luck.

##### Student Name:
##### Student ID:
##### Date

# 1. Code snippet 
#### (you do not need to change it)

In [90]:
# -*- coding: utf-8 -*-
#Imports
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import sys, gzip, json, datetime
import nltk
import random
import numpy as np
import re
import string 
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import vstack


In [169]:
#Opening the Dataset
dataset= pd.read_csv("mpi_snopes.csv",encoding="utf-8")
print len(dataset)
dataset.head()

4713


,Unnamed: 0,Claim,Claim_ID,Credibility,Description,Example,Fact Check,Google Results,Last Updated,Originally Published,Origins,Referred Links,Tags,URL
0,0,"Meijer is offering $100 off ""Back to School"" c...",100-meijer-coupon,false,While each of these scams feature slight varia...,"[Collected via e-mail, August 2015] Meijer cou...","Is Meijer offering $100 off ""Back to School"" c...","[{u'results': [{u'domain': u'www.mlive.com', u...",24 August 2015,24 August 2015,"In August 2015, a survey scam promising custom...",https://www.facebook.com/meijer/photos/pb.9025...,meijer;meijer coupon;coupon;coupon scam;survey...,www.snopes.com/100-meijer-coupon/
1,1,"A photograph depicts a real (giant) pizza, par...",1000-pizza-challenge,true,"However, that linked image led only to a ""comi...","[Collected via e-mail, August 2015] I've seen ...","Does a photograph show a real (giant) pizza, p...","[{u'results': [{u'domain': u'www.snopes.com', ...",5 August 2015,5 August 2015,"In August 2015, the above-displayed image (of ...",http://www.bigmamaspizza.com/franchisee/coming...,pizza;giant pizza;pizza challenge;competitive ...,www.snopes.com/1000-pizza-challenge/
2,2,Image depicts a 102 lb. shrimp caught near Hom...,102-lb-shrimp,false,The image shows a man proudly posing with gian...,"[Collected via Facebook, November 2015]",NaN,"[{u'results': [{u'domain': u'www.snopes.com', ...",27 November 2015,27 November 2015,"On 6 November 2015, a Facebook user shared the...",http://www.livescience.com/47718-giant-alien-s...,102 lb shrimp;homosassa;fishing;,www.snopes.com/102-lb-shrimp/
3,3,A 14-year-old girl became pregnant after recei...,14-year-old-pregnant-flu-shot,false,She had all the typical symptoms of a pregnant...,"[Collected via e-mail, October 2015] I have co...",Did a 14-year-old girl become pregnant after r...,[{u'results': [{u'domain': u'worldnewsdailyrep...,5 October 2015,5 October 2015,On 23 September 2015 the web site World News D...,NaN,fake news;world news daily report;WNDR;flu sho...,www.snopes.com/14-year-old-pregnant-flu-shot/
4,4,NASA confirmed there will be 15 straight days ...,15-days-darkness-november,false,Astronomers from NASA have indicated that the ...,"[Collected via e-mail, December 2015] I can't ...",Has NASA confirmed that there will be 15 strai...,"[{u'results': [{u'domain': u'newswatch33.com',...",4 December 2015,22 July 2015,"On 14 July 2015, the web site Newswatch33 publ...",http://newswatch28.com/;https://www.nasa.gov/a...,nasa;newswatch33;huzlers;15 days of darkness;,www.snopes.com/15-days-darkness-november/


### Bank of useful functions


In [198]:
#creating a SVM model
y = dataset.Credibility.values
X_text = dataset.Claim.values
tfidfVectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, analyzer='word',stop_words='english')
tfidfVectorizer.fit(X_text)
X = tfidfVectorizer.transform(X_text) 

model = svm.SVC(kernel='linear', C=1,probability=True).fit(X, y)

def predictCredibility(claim,model,tfidfVectorizer):
    prediction_score=np.max(model.predict_proba(tfidfVectorizer.transform([claim])))
    prediction=model.predict(tfidfVectorizer.transform([claim]))[0]
    return [prediction,prediction_score]
print predictCredibility("Global warming has stopped",model,tfidfVectorizer)


def getMostSimiarClaims(claim,tfidfVectorizer,X_text,X,quant):
    vectors_ = vstack((tfidfVectorizer.transform([claim]),X))
    #np.append(tfidfVectorizer.transform([claim]),X)
    similirity=[]
    for v in vectors_:
        #computing cosine similarity
        similirity.append(cosine_similarity(vectors_[0],v)[0][0])
    index = np.array(similirity).argsort()[-quant-1:][::-1][-quant:]
    return zip(index,X_text[index],np.array(similirity)[index])
print getMostSimiarClaims("Global warming has stopped",tfidfVectorizer, X_text,X,5)

[u'false', 0.78023753585136235]
[(3888, u'The U.S. government is opening gas stations to distribute free gasoline in poor neighborhoods.', 0.51882202323508442), (4099, u'Florida motorists can block the DMV from disclosing their motor vehicle and driver license records.', 0.48975879258509247), (1663, u'Urban Outfitters offered an item appearing to be a bloodstained Kent State sweatshirt for sale.', 0.21190524362869273), (3172, u'Chrysler dealerships owned by Republican donors were disproportionately targeted for closure at the behest of the Obama administration.', 0.19708275992147156), (1835, u'Someone who has taken LSD more than seven times is automatically deemed legally insane.', 0.1737216891217683)]


# 2. Create 'FakeNews' class
#### This is the part that you need to create the class 'FakeNews':

In [ ]:
#using the "bank of function" above, create a class "FakeNEws" according to the specifications below.
class FakeNEws:
    

# 3. Test 'FakeNews' class
### You do not need to edit this part. The class that you created has to be able to return a valid answer to the following methods:

In [ ]:
fake_news_detector = FakeNEws()
fake_news_detector.setModel(model)
print fake_news_detector.predictCredibilityOfClaim("Hurricane Harvey brought a shark ashore in Houston")
print fake_news_detector.getTopKMostRelatedArticles(5)
